In [1]:
import pandas as pd
import numpy as np

squads = pd.read_csv("../../data-csv/squads.csv")
players = pd.read_csv("../../data-processed/players_clean.csv")
rankings = pd.read_csv("../../data-processed/ranking_clean.csv")

In [2]:
# ------------------------------------------------------
# 2. Filter squads for valid tournaments
# ------------------------------------------------------
valid_tournaments = [
    "WC-2002", "WC-2006", "WC-2010",
    "WC-2014", "WC-2018", "WC-2022"
]
squads = squads[squads["tournament_id"].isin(valid_tournaments)].reset_index(drop=True)

In [3]:
# ------------------------------------------------------
# 3. Team name consistency
# ------------------------------------------------------
# Need team names to match between squads and rankings
def clean_team_name(x):
    if not isinstance(x, str):
        return x
    x = x.strip().title()
    if x == "Iran":
        return "IR Iran"
    if x == "South Korea":
        return "Korea Republic"
    if x == "North Korea":
        return "Korea DPR"
    if x == "China":
        return "China PR"
    if x == "USA":
        return "United States"
    return x

squads["team_name"] = squads["team_name"].apply(clean_team_name)
rankings["country_full"] = rankings["country_full"].apply(clean_team_name)

In [4]:
# ------------------------------------------------------
# 4. Prepare for merge (create 'year' key)
# ------------------------------------------------------
# Extract year from tournament_id (e.g., "WC-2002" -> "2002")
squads["year"] = squads["tournament_id"].str.extract(r'(\d{4})')

# Extract year from rank_date (e.g., "2002-05-15" -> "2002")
rankings["year"] = pd.to_datetime(rankings["rank_date"]).dt.year.astype(str)

In [5]:
# ------------------------------------------------------
# 5. Merge data to calculate age
# ------------------------------------------------------
# Link squads to Players (to get birth_date)
# Use 'left' to keep squad info even if player birth_date is missing (though clean csv should have them)
squad_stats = squads.merge(
    players[["player_id", "birth_date"]], 
    on="player_id", 
    how="left"
)

# Link Squads to Rankings (to get rank_date)
# Need the specific rank_date to calculate the exact age
squad_stats = squad_stats.merge(
    rankings[["country_full", "year", "rank_date"]],
    left_on=["team_name", "year"],
    right_on=["country_full", "year"],
    how="inner"
)

In [7]:
# ------------------------------------------------------
# 6. Calculate median age
# ------------------------------------------------------
# Convert dates
squad_stats["birth_date"] = pd.to_datetime(squad_stats["birth_date"], errors="coerce")
squad_stats["rank_date"] = pd.to_datetime(squad_stats["rank_date"], errors="coerce")

# Calculate age in years (using 365.25 for leap year adjustment)
squad_stats["age"] = (squad_stats["rank_date"] - squad_stats["birth_date"]).dt.days / 365.25

# Compute median age per team per tournament
# Group by country and year, then take the median of the 'age' column
median_ages = squad_stats.groupby(["country_full", "year"])["age"].median().reset_index()
median_ages.rename(columns={"age": "median_age"}, inplace=True)

# Round to 1 decimal place
median_ages["median_age"] = median_ages["median_age"].round(1)

In [8]:
# ------------------------------------------------------
# 7. Create final "squads_clean" (Rankings + median age)
# ------------------------------------------------------
# Start with original rankings
final_df = rankings.copy()

# Merge in the calculated median age
final_df = final_df.merge(
    median_ages,
    on=["country_full", "year"],
    how="left"
)

# Drop the helper 'year' column if you want to keep it strictly like ranking_clean
final_df = final_df.drop(columns=["year"])

final_df.to_csv("../../data-processed/squads_clean.csv", index=False)
print("Saved → ../../data-processed/squads_clean.csv")

Saved → ../../data-processed/squads_clean.csv
